# Further Classification

In this notebook we will look at further aspects of classification, including performing a robust evaluation of a classifier.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
# imports for plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

For this example, we will use the [*Diabetes* dataset](https://www.kaggle.com/datasets/mathchi/diabetes-data-set), originally collected by the US National Institute of Diabetes. Given a set of patient diagnostic measurements, the objective of this task is to predict whether or not the patient has diabetes. The features in the data are as follows:

- *preg*: Number of times the patient has been pregnant
- *plasma*: Plasma glucose concentration
- *pres*: Diastolic blood pressure
- *skin*: Triceps skin fold thickness
- *insulin*: 2-Hour serum insulin level
- *bmi*: Body mass index 
- *family*: Score likelihood of diabetes based on family history
- *age*: The patient's age in years
- *outcome*: The actual diagnosis for the patient ('positive' or 'negative')


## Data Loading and Preparation

Load the complete dataset from a CSV file:

In [ ]:
df = pd.read_csv('diabetes.csv')
df.head()

Look at the distribution of the target label 'outcome' in the data:

In [ ]:
df["outcome"].value_counts()

Separate out the features to use for classification from the target label itself:

In [ ]:
target = df["outcome"].values
other_columns = list(df.columns)
other_columns.remove("outcome")
data = df[other_columns]
data.head()

Finally, we will apply normalisation to the numeric data. In this case, the variables are mean centered and scaled by the standard deviation:

In [ ]:
normalizer = StandardScaler()
data_scaled = normalizer.fit_transform(data.values)
data_scaled

## Evaluation Metrics

We will now look at different ways to evaluate a classification algorithm. We will focus on the case of *binary classification* (two classes) using the diabetes data, although these methods generalise to *multi-class* problems.

As we've seen previously, we can easily randomly split the complete dataset into a training test and a test set. We will specify that 40% (0.4) of the data will be used for the test set. The remaining 60% will be used to train the model.

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(data_scaled, target, test_size=0.4)
print("Training set has %d examples" % data_train.shape[0])
print("Test set has %d examples" % data_test.shape[0])

Now we will build a KNN classifier (*k=3* neighbours) based on the training data:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
m = knn.fit(data_train, target_train)

Make our predictions on the test set and evaluate them using the standard classification *accuracy* score:

In [ ]:
predicted = knn.predict(data_test)
# calculate the accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_test, predicted)
print("Accuracy=%.3f" % acc)

Accuracy scores only provide us with a single score summarising a classifier's performance. Often we want to understand where a classifier performs well or poorly in more detail.

In particular, we might want to determine the extent to which the classifier made the following correct/incorrect predictions:
- *True Positives* (TP) are those which are labeled 'positive' which are actually 'positive'
- *False Positives* (FP) are those which are labeled 'positive' which are actually 'negative'
- *True Negatives* (TN) are those which are labeled 'negative' which are actually 'negative'
- *False Negatives* (FN) are those which are labeled 'negative' which are actually 'positive'

We can do this by creating a confusion matrix for the results. The result is a NumPy matrix, with predictions on the columns and actual labels on the rows. The values correspond to:

    [ [TP FN]
    [FP TN] ]
A perfect classifier with 100% accuracy would produce a pure diagonal matrix which would have all the test examples predicted in their correct class. In our case, we see that we have many false negatives (i.e. examples labelled -1 which are actually 1).

In [ ]:
from sklearn.metrics import confusion_matrix
# build the confusion matrix
cm = confusion_matrix(target_test, predicted)
print(cm)

We can produce a more visual version of this matrix using the *ConfusionMatrixDisplay* class:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
# note we get the labels (names) for the classes from the classifier
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn.classes_)
disp.plot();

Measures from information retrieval (search engines) can be used in ML evaluation. Note that these are calculated with respect to a particular class (e.g. positive class or the negative class):
- *Precision*: proportion of retrieved results that are relevant = TP/(TP+FP)
- *Recall*: proportion of relevant results that are retrieved = TP/(TP+FN)

In [ ]:
from sklearn.metrics import precision_score, recall_score
# indicate that we are interested in the Positive class here
print("Precision(Positive) = %.3f" % precision_score(target_test, predicted, pos_label="positive") )
print("Recall(Positive) = %.3f" % recall_score(target_test, predicted, pos_label="positive") )

In [ ]:
# alternatively indicate that we are interested in the Negative class here
print("Precision(Negative) = %.3f" % precision_score(target_test, predicted, pos_label="negative") )
print("Recall(Negative) = %.3f" % recall_score(target_test, predicted, pos_label="negative") )

Note that there is often a trade-off between precision and recall. We can combine precision and recall into a single score using the *F1 Measure*, which is a weighted average of the precision and recall. The F1 Measure reaches its best value at 1 and worst at 0. Again this score is calculate with respect to a specified class (e.g. positive or negative)

    F1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
from sklearn.metrics import f1_score
print("F1(Positive) = %.3f" % f1_score(target_test, predicted, pos_label="positive") )
print("F1(Negative) = %.3f" % f1_score(target_test, predicted, pos_label="negative") )

We can quickly compute a summary of these statistics using scikit-learn's provided convenience function:

In [ ]:
from sklearn.metrics import classification_report
# note we get the labels (names) for the classes from the classifier
print(classification_report(target_test, predicted, target_names=knn.classes_))

## Cross Validation

A problem with simply randomly splitting a dataset into two sets is that each random split might give different results. We are also ignoring a portion of your dataset. One way to address this is to use *K-fold cross-validation* to evaluate a classifier:
1. Divide the data into K disjoint subsets - “folds” (e.g. K=5 folds).
2. For each of K experiments, use K-1 folds for training and the selected one fold for testing.
3. Repeat for all K folds, average the accuracy/error rates.

While this is a relatively complex process, scikit-learn allows us to achieve this using a single function *cross_val_score()*. Let's do a 2-fold cross-validation of the KNN classifier

In [ ]:
# create a single classifier
knn = KNeighborsClassifier(n_neighbors=3)
# apply 2-fold cross-validation, measuring accuracy each time
from sklearn.model_selection import cross_val_score
# the argument 'cv' specifies the number of folds to use
acc_scores = cross_val_score(knn, data_scaled, target, cv=2, scoring="accuracy")
print(acc_scores)

Similarly, for 5-fold cross validation we get an array with 5 accuracy scores, one score for each fold:

In [ ]:
acc_scores = cross_val_score(knn, data_scaled, target, cv=5, scoring="accuracy")
print(acc_scores)

Normally we calculate the average accuracy across all folds:

In [ ]:
print("KNN (k=3): Mean cross-validation accuracy = %.3f" % acc_scores.mean())

We can use this approach to compare different classification algorithms on the same data, such as a logistic regression classifier or a Support Vector Machine (SVM) classifier.

In [ ]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(solver='lbfgs')
acc_scores = cross_val_score(lr, data_scaled, target, cv=5, scoring="accuracy")
print("Logistic Regression: Mean cross-validation accuracy = %.3f" % acc_scores.mean())

In [ ]:
from sklearn.svm import SVC
svm = SVC(gamma='auto')
acc_scores = cross_val_score(svm, data_scaled, target, cv=5, scoring="accuracy")
print("SVM: Mean cross-validation accuracy = %.3f" % acc_scores.mean() )

Finally we could use cross-validation as part of an approach to select parameters for a classifcation algorithm, such as the number of neighbours *k* for a KNN classifier:

In [ ]:
# iterate over a range of values of k
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    acc_scores = cross_val_score(knn, data_scaled, target, cv=5, scoring="accuracy")
    mean_acc = acc_scores.mean()
    print("K=%02d neighbours: Accuracy=%.3f" % (k, mean_acc))